   # Library


In [1]:
! pip install sentencepiece einops wandb torch-summary icecream -qq

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pprint import pprint
from nltk.tokenize import word_tokenize as en_tokenizer
import sentencepiece as spm
import urllib.request
import csv
import numpy as np
from einops import rearrange, reduce, repeat
from torch.cuda import amp
from tqdm import tqdm
import wandb
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os
from icecream import ic
from sklearn.model_selection import train_test_split
import os




In [11]:
TRAIN_PATH = './train'
VOCAB_SIZE = 10000
SEQ_LEN = 60


PAD_IDX = 0
BOS_IDX = 2
EOS_IDX = 3

WANDB_LOAD_PATH = 'jiwon7258/Transformer_enko/z0rsx1lp'


# ENV = 'COLAB'
ENV = 'KAGGLE'
# ENV = 'SYSTEM'

SAVE = True
LOAD = False
# SAVE = False
# LOAD = True

# Option for Mixed Precision
FP16 = True
# FP16 = False

N = 3
HIDDEN_DIM = 128
NUM_HEAD = 8 
INNER_DIM = 256
BATCH_SIZE = 32

CONFIG = {
    'VOCAB_SIZE': VOCAB_SIZE,
    'SEQ_LEN': SEQ_LEN,
    'N': N,
    'HIDDEN_DIM': HIDDEN_DIM,
    'NUM_HEAD': NUM_HEAD,
    'INNER_DIM': INNER_DIM,
    'BATCH_SIZE': BATCH_SIZE,
}


if 'device' not in globals():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device}')


Using cpu


In [10]:
import wandb
import os
# if want to run in offline mode

# os.environ["WANDB_MODE"] = "dryrun"
# wandb.init(project="Transformer_bible", entity="jiwon7258")


os.environ["WANDB_MODE"] = "online"
wandb.init(project="Transformer_bible", entity="jiwon7258", config = CONFIG, job_type='data')
wandb.run.name = f"dataset_{VOCAB_SIZE}_{SEQ_LEN}_{N}_{HIDDEN_DIM}_{INNER_DIM}"

dataset = wandb.Artifact(f'bible-dataset_{VOCAB_SIZE}_{SEQ_LEN}', type='dataset')



In [5]:
if (ENV == 'COLAB'):
    from google.colab import drive
    drive.mount('/content/drive')


In [13]:
DATASET_PATH = './'
if (ENV == 'KAGGLE') :
    DATASET_PATH = '../input/enko-bible'
elif (ENV == 'COLAB') :
    DATASET_PATH = '/content/drive/MyDrive/notebooks/transformer_bible/'


   # 영어 데이터 로드

In [14]:
en_train = open(os.path.join(DATASET_PATH, 'bible-all.en.txt'))
en_train_content = en_train.read()


In [16]:
en_train_list = en_train_content.split('\n')


   # 한국어 데이터 로드

In [18]:
ko_train = open(os.path.join(DATASET_PATH, 'bible-all.kr.txt'))
ko_train_content = ko_train.read()

In [19]:
ko_train_list = ko_train_content.split('\n')

In [20]:
en_train_list[:10]

['Genesis1.1  In the beginning God created the heavens and the earth.',
 'Genesis1.2  Now the earth was formless and empty, darkness was over the surface of the deep, and the Spirit of God was hovering over the waters.',
 'Genesis1.3  And God said, "Let there be light," and there was light.',
 'Genesis1.4  God saw that the light was good, and He separated the light from the darkness.',
 'Genesis1.5  God called the light "day," and the darkness he called "night." And there was evening, and there was morning--the first day.',
 'Genesis1.6  And God said, "Let there be an expanse between the waters to separate water from water."',
 'Genesis1.7  So God made the expanse and separated the water under the expanse from the water above it. And it was so.',
 'Genesis1.8  God called the expanse "sky." And there was evening, and there was morning--the second day.',
 'Genesis1.9  And God said, "Let the water under the sky be gathered to one place, and let dry ground appear." And it was so.',
 'Genes

   # DATA 데이터프레임

In [21]:
data = pd.DataFrame()
data['en_raw'] = en_train_list
data['ko_raw'] = ko_train_list

In [23]:
data.head()

,en_raw,ko_raw
0,Genesis1.1 In the beginning God created the h...,Genesis1.1 태초에 하나님이 천지를 창조하셨다.
1,Genesis1.2 Now the earth was formless and emp...,"Genesis1.2 땅이 혼돈하고 공허하며, 어둠이 깊음 위에 있고, 하나님의 영..."
2,"Genesis1.3 And God said, ""Let there be light,...","Genesis1.3 하나님이 말씀하시기를 ""빛이 생겨라"" 하시니, 빛이 생겼다."
3,"Genesis1.4 God saw that the light was good, a...","Genesis1.4 그 빛이 하나님 보시기에 좋았다. 하나님이 빛과 어둠을 나누셔서,"
4,"Genesis1.5 God called the light ""day,"" and th...","Genesis1.5 빛을 낮이라고 하시고, 어둠을 밤이라고 하셨다. 저녁이 되고 ..."


In [24]:
len(data)


31104

In [25]:
data = data.reset_index(drop = True)
data.head()

,en_raw,ko_raw
0,Genesis1.1 In the beginning God created the h...,Genesis1.1 태초에 하나님이 천지를 창조하셨다.
1,Genesis1.2 Now the earth was formless and emp...,"Genesis1.2 땅이 혼돈하고 공허하며, 어둠이 깊음 위에 있고, 하나님의 영..."
2,"Genesis1.3 And God said, ""Let there be light,...","Genesis1.3 하나님이 말씀하시기를 ""빛이 생겨라"" 하시니, 빛이 생겼다."
3,"Genesis1.4 God saw that the light was good, a...","Genesis1.4 그 빛이 하나님 보시기에 좋았다. 하나님이 빛과 어둠을 나누셔서,"
4,"Genesis1.5 God called the light ""day,"" and th...","Genesis1.5 빛을 낮이라고 하시고, 어둠을 밤이라고 하셨다. 저녁이 되고 ..."


In [32]:
data['en'] = data['en_raw'].apply(lambda x: x.split(' ')[1:])
data['en'] = data['en'].apply(lambda x: (' ').join(x))
data['ko'] = data['ko_raw'].apply(lambda x: x.split(' ')[1:])
data['ko'] = data['ko'].apply(lambda x: (' ').join(x))


In [39]:
data = data[['en','ko']]
data.head()

,en,ko
0,In the beginning God created the heavens and ...,태초에 하나님이 천지를 창조하셨다.
1,"Now the earth was formless and empty, darknes...","땅이 혼돈하고 공허하며, 어둠이 깊음 위에 있고, 하나님의 영은 물 위에 움직이고..."
2,"And God said, ""Let there be light,"" and there...","하나님이 말씀하시기를 ""빛이 생겨라"" 하시니, 빛이 생겼다."
3,"God saw that the light was good, and He separ...","그 빛이 하나님 보시기에 좋았다. 하나님이 빛과 어둠을 나누셔서,"
4,"God called the light ""day,"" and the darkness ...","빛을 낮이라고 하시고, 어둠을 밤이라고 하셨다. 저녁이 되고 아침이 되니, 하루가..."


   # Train Sentencepiece

In [41]:
with open('src.txt', mode = 'w', encoding='utf8') as f:
    f.write('\n'.join(data['en']))
with open('trg.txt', mode= 'w', encoding='utf8') as f:
    f.write('\n'.join(data['ko']))




In [45]:
%%capture context
corpus = "src.txt"
prefix = "src"
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={VOCAB_SIZE}" +
    " --model_type=bpe" +
    " --max_sentence_length=999999" +  # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" +  # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" +  # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]");  # 사용자 정의 토큰





In [46]:
%%capture context
corpus = "trg.txt"
prefix = "trg"
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={VOCAB_SIZE}" +
    " --model_type=bpe" +
    " --max_sentence_length=999999" +  # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" +  # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" +  # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]");  # 사용자 정의 토큰





# 정수 인코딩

In [48]:
sp_src = spm.SentencePieceProcessor()
sp_src.Load('src.model')


for idx in range(3):
    sentence = data['en'][idx]
    print(sp_src.EncodeAsPieces(sentence))
    print(sp_src.EncodeAsIds(sentence))




['▁In', '▁the', '▁beginning', '▁God', '▁created', '▁the', '▁heavens', '▁and', '▁the', '▁earth', '.']
[502, 10, 2155, 133, 3212, 10, 1354, 19, 10, 458, 9961]
['▁Now', '▁the', '▁earth', '▁was', '▁form', 'less', '▁and', '▁empty', ',', '▁darkness', '▁was', '▁over', '▁the', '▁surface', '▁of', '▁the', '▁deep', ',', '▁and', '▁the', '▁Spirit', '▁of', '▁God', '▁was', '▁ho', 'vering', '▁over', '▁the', '▁waters', '.']
[589, 10, 458, 127, 3464, 636, 19, 3330, 9958, 1451, 127, 268, 10, 6810, 21, 10, 1685, 9958, 19, 10, 837, 21, 133, 127, 386, 8186, 268, 10, 1411, 9961]
['▁And', '▁God', '▁said', ',', '▁"', 'Let', '▁there', '▁be', '▁light', ',"', '▁and', '▁there', '▁was', '▁light', '.']
[288, 133, 150, 9958, 65, 1612, 250, 52, 897, 393, 19, 250, 127, 897, 9961]


In [49]:
def en_encode(tmpstr:str) -> np.array :
    tmpstr = np.array(sp_src.EncodeAsIds(tmpstr))

    # SEQ_LEN보다 길면 짜른다 
    if len(tmpstr) > SEQ_LEN :
        tmpstr = tmpstr[:SEQ_LEN]

    # SEQ_LEN보다 작으면 padding
    else :
        tmpstr = np.pad(tmpstr, (0, SEQ_LEN - len(tmpstr)), 'constant', constant_values = sp_src.pad_id())
    
    return tmpstr




In [50]:
# src_data는 data['en']를 참조한다. (동일 id)
src_data = data['en']

src_list = []

for idx in range(len(src_data)):
    src_list.append(en_encode(src_data[idx]))

src_list[0]




array([ 502,   10, 2155,  133, 3212,   10, 1354,   19,   10,  458, 9961,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

In [51]:
sp_trg = spm.SentencePieceProcessor()
sp_trg.Load('trg.model')


for idx in range(3):
    sentence = data['ko'][idx]
    print(sp_trg.EncodeAsPieces(sentence))
    print(sp_trg.EncodeAsIds(sentence))




['▁태', '초', '에', '▁하나님이', '▁천', '지를', '▁창조', '하셨다', '.']
[561, 9349, 8964, 213, 369, 513, 2208, 883, 8962]
['▁땅이', '▁혼', '돈', '하고', '▁공', '허', '하며', ',', '▁어둠이', '▁깊', '음', '▁위에', '▁있고', ',', '▁하나님의', '▁영', '은', '▁물', '▁위에', '▁움직', '이고', '▁계셨다', '.']
[1226, 1567, 9398, 106, 440, 9291, 455, 8961, 4716, 1114, 9043, 394, 696, 8961, 194, 153, 8978, 119, 394, 5214, 411, 4486, 8962]
['▁하나님이', '▁말씀하시기를', '▁"', '빛이', '▁생겨', '라', '"', '▁하시니', ',', '▁빛이', '▁생', '겼다', '.']
[213, 2045, 32, 7888, 5865, 8983, 8995, 2921, 8961, 3057, 171, 1450, 8962]


In [52]:
def ko_encode(tmpstr: str) -> np.array:
    tmpstr = np.array(sp_trg.EncodeAsIds(tmpstr))
    tmpstr = np.insert(tmpstr, 0, sp_trg.bos_id())

    if len(tmpstr) >= SEQ_LEN:
        # SEQ_LEN -1의 길이로 자른다
        tmpstr = tmpstr[:SEQ_LEN-1]
        # 마지막에 <eos> 토큰을 넣어줌으로써, 길이를 SEQ_LEN으로 맞춘다
        tmpstr = np.pad(tmpstr, (0, 1),
                        'constant', constant_values=sp_trg.eos_id())


    else:
        tmpstr = np.pad(tmpstr, (0, 1),
                        'constant', constant_values=sp_trg.eos_id())
        tmpstr = np.pad(tmpstr, (0, SEQ_LEN - len(tmpstr)),
                        'constant', constant_values=sp_trg.pad_id())

    return tmpstr





In [53]:
# trg_data는 data['ko']를 참조한다. (동일 id)
trg_data = data['ko']

trg_list = []

for idx in range(len(trg_data)):
    trg_list.append(ko_encode(trg_data[idx]))   

trg_list[0]




array([   2,  561, 9349, 8964,  213,  369,  513, 2208,  883, 8962,    3,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

   # Train Test Split

In [54]:
src_train, src_valid, trg_train, trg_valid = train_test_split(src_list, trg_list, test_size=0.2, random_state=42)

  # Save

In [ ]:
if (SAVE):
    # Save data to local
    import joblib

    joblib.dump(src_train, 'src_train.pkl')
    joblib.dump(trg_train, 'trg_train.pkl')
    joblib.dump(src_valid, 'src_valid.pkl')
    joblib.dump(trg_valid, 'trg_valid.pkl')
    joblib.dump(src_list, 'src_list.pkl')
    joblib.dump(trg_list, 'trg_list.pkl')

    # save data in wandb
    names = [
        'src_train',
        'src_valid',
        'trg_train',
        'trg_valid',
        'src_list',
        'trg_list', ]

    for name in names:
        dataset.add_file(f'{name}.pkl')

    names = [
        'src.model',
        'trg.model'
    ]
    for name in names:
        dataset.add_file(f'{name}')


        




In [ ]:
wandb.run.log_artifact(dataset)
wandb.finish()